### Steps to deploy a minimal ML model with prefect and coiled.  
#### Pramble 
In this example we will use anaconda for our python enviroment manager.  Other python enviroments can also be used, but there is an extra step in the python enviroment installation process: `dask` and `dask-distributed` need to be installed seperately when using pip.

Furthermore you will need to sign up for a coiled account and prefect account.  As of this post, (2021-05-04), `coiled` is free. And for these purposes so is `prefect`.


 
#### 1. In the shell create a new enviroment with `prefect`, `dask`, `coiled`. 

In [ ]:
`conda create --name py38Pcoiled python=3.8 dask coiled prefect dask-ml -c conda-forge -c conda`

- If you are wondering why the python version is being typed out in specifity.  It is because the local and cloud enviroment need to match exactly.  Otherwise all sort of cloud pickle errors occur.  In a later step we will add versions for a couple of other crucial packeges .e.g. `coiled`

#### 2. Add your coiled token

a. In the`coiled` UI navigate to the **Dashboard** -> **Quick Sart**

![Coiled Token](assets/coiled-login.png)

b. Copy `coiled login --token ·········································` to the shell and run.



#### 3. Create an enviroment requirements file.  

There are four options to specify the python enviroment to run the code. Conda, Pip, container and post_build.   It's possible to use all four or any combination:  Yes you are reading this correctly.  There is no need for a dockerfile.
You can name it whatever you want. 

In this case we will use a conda enviroment

In [ ]:
conda env export > <py38mlflow.yml>







It's mot that coiled doesnt use one to execute the flow, but rather it merges your specifications with theirs.  The 






#### 4. Create a software enviroment to run in the cloud

- 


In [7]:
import coiled

In [13]:

coiled.create_software_environment(
    # name is the software enviroment name in coiled.  i.e : "<account>/<name>"
    name="gryBox/py38Pcoiled", 
    conda="/home/ilivni/demos/blogs/coiled-prefect-demo/py38mlflow.yml"
    
)


Updating software environment...
Creating new software environment
Creating new ecr build
STEP 1: FROM coiled/default:sha-050a418
STEP 2: COPY environment.yml environment.yml
--> e8c68dd0d87
STEP 3: RUN conda env update -n coiled -f environment.yml     && rm environment.yml     && conda clean --all -y     && echo "conda activate coiled" >> ~/.bashrc
Solving environment: ...working... done

sortedcontainers-2.3 | 26 KB     | ########## | 100%
aiobotocore-1.3.0    | 40 KB     | ########## | 100%
zict-2.0.0           | 10 KB     | ########## | 100%
mypy_extensions-0.4. | 10 KB     | ########## | 100%
libgcc-ng-9.3.0      | 7.8 MB    | ########## | 100%
prompt-toolkit-3.0.1 | 244 KB    | ########## | 100%
xz-5.2.5             | 343 KB    | ########## | 100%
rich-10.1.0          | 333 KB    | ########## | 100%
boto3-1.17.49        | 70 KB     | ########## | 100%
jpeg-9d              | 264 KB    | ########## | 100%
pyopenssl-20.0.1     | 48 KB     | ########## | 100%
tk-8.6.10            | 3

In [ ]:
from ml_flow import flow

In [11]:
from prefect.executors import DaskExecutor

executor = DaskExecutor(
    cluster_class=coiled.Cluster,
    cluster_kwargs={
        "software": "grybox/py38pcoiled",
        "shutdown_on_close": False,
        "name": "prefect-play",
    },
)


flow.run(
    executor=executor
)

[2021-05-06 23:57:41-0500] INFO - prefect.FlowRunner | Beginning Flow run for 'ml-flow'
[2021-05-06 23:57:41-0500] INFO - prefect.DaskExecutor | Creating a new Dask cluster with `coiled.cluster.Cluster`...


Output()

Task was destroyed but it is pending!
task: <Task pending name='Task-298' coro=<Cloud._create_software_environment() running at /home/ilivni/miniconda3/envs/py38Pcoiled/lib/python3.8/site-packages/coiled/core.py:1013> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x7f0c582e4550>()]> cb=[IOLoop.add_future.<locals>.<lambda>() at /home/ilivni/miniconda3/envs/py38Pcoiled/lib/python3.8/site-packages/tornado/ioloop.py:688]>


tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <zmq.eventloop.ioloop.ZMQIOLoop object at 0x7f0c53576850>>, <Future finished exception=AssertionError()>)
Traceback (most recent call last):
  File "/home/ilivni/miniconda3/envs/py38Pcoiled/lib/python3.8/site-packages/tornado/ioloop.py", line 741, in _run_callback
    ret = callback()
  File "/home/ilivni/miniconda3/envs/py38Pcoiled/lib/python3.8/site-packages/tornado/ioloop.py", line 765, in _discard_future_result
    future.result()
  File "/home/ilivni/miniconda3/envs/py38Pcoiled/lib/python3.8/site-packages/tornado/gen.py", line 775, in run
    yielded = self.gen.send(value)
  File "/home/ilivni/miniconda3/envs/py38Pcoiled/lib/python3.8/site-packages/distributed/utils.py", line 340, in f
    assert thread_state.asynchronous > 0
AssertionError


KeyboardInterrupt: 